In [29]:
%load_ext sql
import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [30]:
pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# pd.read_sql用のコネクタ
conn = psycopg2.connect(**pgconfig)
# pd.to_sql用のcreate engine
engine = create_engine(dsl)
# MagicコマンドでSQLを書くための設定
%sql $dsl

'Connected: padawan@dsdojo_db'

In [31]:
df_calender = pd.read_sql(sql='select * from calender', con=conn)
df_customer = pd.read_sql(sql='select * from customer', con=conn)
df_gender = pd.read_sql(sql='select * from gender', con=conn)
df_prefecture = pd.read_sql(sql='select * from prefecture', con=conn)
df_product = pd.read_sql(sql='select * from product', con=conn)
df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
df_store = pd.read_sql(sql='select * from store', con=conn)

In [32]:
print('df_calender:', len(df_calender))
print('df_customer:', len(df_customer))
print('df_gender:', len(df_gender))
print('df_prefecture:', len(df_prefecture))
print('df_product:', len(df_product))
print('df_receipt:', len(df_receipt))
print('df_store:', len(df_store))

df_calender: 1826
df_customer: 224439
df_gender: 2
df_prefecture: 48
df_product: 10022
df_receipt: 1100
df_store: 53


In [33]:
df_calender.tail()

,ymd,year,quater,half_period,ym,year_weeki,month,day,day_of_the_week
1821,2011-12-27,2011,20114,20112,201112,201152,12,27,火
1822,2011-12-28,2011,20114,20112,201112,201152,12,28,水
1823,2011-12-29,2011,20114,20112,201112,201152,12,29,木
1824,2011-12-30,2011,20114,20112,201112,201152,12,30,金
1825,2011-12-31,2011,20114,20112,201112,201152,12,31,土


-> カレンダーは不要

In [34]:
df_customer.tail()

,customer_id,surname,first_name,surname_kana,first_name_kana,gender_cd,birth_day,prefecture_cd,post_cd,address,address_kana,tel_no,join_store_cd,join_date
224434,CS053715000002,岡本,千穂,オカモト,チホ,1,1936-10-17,12,279-0023,千葉県浦安市高洲三丁目13,チバケンウラヤスシタカスサンチョウメ,047-123-4567,S12053,2009-04-02
224435,CS053715000003,伊藤,朋,イトウ,トモ,1,1937-08-19,12,279-0013,千葉県浦安市日の出二丁目14,チバケンウラヤスシヒノデニチョウメ,047-123-4567,S12053,2009-04-04
224436,CS053715000004,八重,美穂,ヤエ,ミホ,1,1933-09-25,12,279-0024,千葉県浦安市港54,チバケンウラヤスシミナト,047-123-4567,S12053,2009-04-02
224437,CS053813000001,長谷川,麻依,ハセガワ,マイ,1,1920-12-30,12,279-0023,千葉県浦安市高洲三丁目21,チバケンウラヤスシタカスサンチョウメ,047-123-4567,S12053,2009-04-22
224438,ZZ000000000000,非会員,NULL,ヒカイイン,NULL,9,None,99,None,None,None,None,999,2000-09-07


-> 名称（姓名の漢字、カナ）不要、住所（漢字、カナ）不要、電話番号不要

In [35]:
df_customer = df_customer[['customer_id', 'gender_cd', 'birth_day', 'prefecture_cd', 'post_cd', 'join_store_cd', 'join_date']]

In [36]:
df_customer.tail()

,customer_id,gender_cd,birth_day,prefecture_cd,post_cd,join_store_cd,join_date
224434,CS053715000002,1,1936-10-17,12,279-0023,S12053,2009-04-02
224435,CS053715000003,1,1937-08-19,12,279-0013,S12053,2009-04-04
224436,CS053715000004,1,1933-09-25,12,279-0024,S12053,2009-04-02
224437,CS053813000001,1,1920-12-30,12,279-0023,S12053,2009-04-22
224438,ZZ000000000000,9,None,99,None,999,2000-09-07


In [37]:
df_customer['join_date'].max()

datetime.date(2009, 12, 31)

In [38]:
df_customer.dtypes

customer_id      object
gender_cd        object
birth_day        object
prefecture_cd    object
post_cd          object
join_store_cd    object
join_date        object
dtype: object

In [39]:
from dateutil.relativedelta import relativedelta

In [40]:
max(df_customer['join_date'] + relativedelta(years=10))

datetime.date(2019, 12, 31)

In [41]:
df_receipt.tail()

,sales_ymd,store_cd,product_cd,customer_id,receipt_no,receipt_sub_no,amount,quantity
1095,2010-01-05,S14034,P059001026,CS034414000083,1182,1,588,1
1096,2010-01-05,S14033,P071006001,ZZ000000000000,1172,2,108,1
1097,2010-01-05,S14042,P071001018,ZZ000000000000,1182,2,102,1
1098,2010-01-05,S14049,P050402003,ZZ000000000000,1182,2,480,1
1099,2010-01-05,S14045,P091402001,ZZ000000000000,1192,2,900,1


In [42]:
df_receipt['sales_ymd'].max()


datetime.date(2010, 1, 5)

-> 9年シフトまでは行けそう

In [43]:
df_receipt['sales_ymd'] = df_receipt['sales_ymd']+ relativedelta(years=9)

In [45]:
df_receipt['sales_ymd'].min()

datetime.date(2019, 1, 1)

In [46]:
df_receipt['sales_ymd'].max()

datetime.date(2019, 1, 5)